In [ ]:

import os
import shutil
import scipy
import numpy as np
import tensorflow as tf
from tensorflow import keras as ks 

In [ ]:

root_dir = "PetImages" 

training_cats_dir = "PetImages/Training/cats" 
validation_cats_dir = "PetImages/Validation/cats"

training_dogs_dir = "PetImages/Training/dogs" 
validation_dogs_dir = "PetImages/Validation/dogs"

for directory in [training_cats_dir,training_dogs_dir,validation_cats_dir,validation_dogs_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)




In [ ]:

file = os.path.join("PetImages", 'Cat')
print(os.listdir(file))
#print(os.path.getsize(file))
#print(os.listdir(file))

In [ ]:

def split_into_train_validation(root_path, training_set,validation_set, split_size):
    print(root_path)
    files = []
    for filename in os.listdir(root_path):
        file = os.path.join(root_path,filename)
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignorning")
    training_length = int(len(files)*split_size)
    print(training_length)
    shuffled_set = np.random.choice(files, size = len(files),replace= False)
    training = shuffled_set[:training_length]
    validation = shuffled_set[training_length:]

    for filename in training:
        this_file = os.path.join(root_path,filename)
        destination_file = os.path.join(training_set,filename)
        shutil.copyfile(this_file,destination_file)
    
    for filename in validation:
        this_file = os.path.join(root_path,filename)
        destination_file = os.path.join(validation_set,filename)
        shutil.copyfile(this_file,destination_file)



In [ ]:

splitsize = 0.8 #80 data for the validation set
split_into_train_validation(os.path.join(root_dir,'Cat'),training_cats_dir,validation_cats_dir,splitsize)
split_into_train_validation(os.path.join(root_dir,'Dog'),training_dogs_dir,validation_dogs_dir,splitsize)


In [ ]:
def removed_original_directories():
    original_cat_dir = os.path.join("PetImages/Cat/")
    original_dog_dir = os.path.join("PetImages/Dog/")
    if os.path.exists(original_cat_dir):
        shutil.rmtree(original_cat_dir)
    else:
        print("cat directory already removed!")

    if os.path.exists(original_dog_dir):
        shutil.rmtree(original_dog_dir)
    else:
        print("Dog directory already removed!")


In [ ]:
def train_validation_generator(training_dir,validation_dir):
    train_data_gen = ks.preprocessing.image.ImageDataGenerator(rescale=1.0/255)
    train_generator = train_data_gen.flow_from_directory(directory= training_dir,
                                                        batch_size = 20,
                                                        class_mode= 'binary',
                                                        target_size=(150,150))

    validation_data_gen = ks.preprocessing.image.ImageDataGenerator(rescale= 1.0/255)
    validation_generator = validation_data_gen.flow_from_directory(directory= validation_dir,
                                                                batch_size= 20,
                                                                class_mode='binary',
                                                                target_size=(150,150))
    
    return train_generator,validation_generator     

In [ ]:
removed_original_directories()
train_generator, validation_generator = train_validation_generator("PetImages/Training/","PetImages/Validation/")

In [ ]:
def create_model():
    model = ks.models.Sequential([
        ks.layers.Conv2D(16,(3,3),activation= 'relu',input_shape = (150,150,3)),
        ks.layers.MaxPooling2D(2,2),
        ks.layers.Conv2D(32,(3,3),activation= 'relu'),
        ks.layers.MaxPooling2D(2,2),
        ks.layers.Conv2D(64,(3,3),activation= 'relu'),
        ks.layers.MaxPooling2D(2,2),

        ks.layers.Flatten(),
        ks.layers.Dense(512,activation='relu'),
        ks.layers.Dense(1,activation='sigmoid')
    ])
    model.compile(optimizer=ks.optimizers.RMSprop(learning_rate=0.001),
                loss = 'binary_crossentropy',
                metrics=['accuracy'])
    return model
    

In [ ]:

model = create_model()
history = model.fit(train_generator,epochs = 5,verbose = 1,validation_data = validation_generator)


In [ ]:
import matplotlib.pyplot as plt
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc))

plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.show()
print("")

plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.show()



In [ ]:

path='1.jpg'
img=ks.utils.load_img(path, target_size=(150, 150))

x=ks.utils.img_to_array(img)
x /= 255
x=np.expand_dims(x, axis=0)
classes = model.predict(x)
plt.imshow(img)
if classes >0.5:
    print(" is a dog")
else:
    print(" is a cat")
